In [3]:
%pip install firebase-admin google-cloud-firestore

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ------ --------------------------------- 2.1/13.3 MB 9.8 MB/s eta 0:00:02
   -------------- ------------------------- 4.7/13.3 MB 10.6 MB/s eta 0:00:01
   -------------------- ------------------- 6.8/13.3 MB 10.5 MB/s eta 0:00:01
   --------------------------- ------------ 9.2/13.3 MB 10.8 MB/s eta 0:00:01
   ----------------------------------- ---- 11.8/13.3 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------------------- -------------------- 2.1/4.3 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [139]:
#!/usr/bin/env python3
import os
import time
import logging
import argparse
from typing import List, Dict
import datetime
import re  

import firebase_admin
from firebase_admin import credentials, firestore

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from firebase_admin import credentials, initialize_app, firestore
from urllib.parse import urljoin

Some Notes:

The Code below scraps data from the home page of https://planitpurple.northwestern.edu and returns just the first 56 events present on the home page

To-Do:
1. Enable Pagination such that it returns all events for that date so long as there is still a next button on the page
2. My design idea was to scrap events weekly. So each week, we scrap events for that day plus 6 days ahead. Such that a can have a scheduler that runs this script every week. I tried implementing this but faced difficulties
3. I didn't save events yet to the database just so I don't fill it with data that I might end up deleting later. We need to do that after 1 and 2 are complete

We don't have to worry about filters here. We will implement that in the front-end since we are sending all events to the database regardless

In [5]:
# ----- Configuration & Logging -----
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [9]:
# ----- Firestore Initialization -----
# Replace FIREBASE_KEY_PATH with the path to your own Firebase json file. Get the json file by going to Project Settings -> 
# Service Accounts -> Generate new private key. It should download a json file

FIREBASE_KEY_PATH = "C:/Users/nebah/OneDrive/Desktop/394-Web-Scrapping/campusloop-4abd3-firebase-adminsdk-fbsvc-1375db176e.json"
cred = credentials.Certificate(FIREBASE_KEY_PATH)
firebase_admin.initialize_app(cred)
db = firestore.client()

In [11]:
# Check to see if database connection was successful

import pandas as pd

docs = db.collection("events").get()     # returns a list of DocumentSnapshot
records = [d.to_dict() for d in docs]
ids     = [d.id         for d in docs]

df = pd.DataFrame(records, index=ids)
df

,LinktoImage,EventName,Description,Location,LinktoTickets,Date,Saved,Time,Category,UserID
0iQK9GjQqpGTq1VyDvAv,https://firebasestorage.googleapis.com/v0/b/ca...,School of Communication Dean E. Patrick Johnso...,Sarah Jessica Parker is an award-winning actor...,Attend in-person at the Josephine Louis Theate...,,2025-04-25,False,17:30,Guest Speakers,user_2w6cXyvhD9M0HmI1TbUoFZQq52O
1iFqSNncSj8Sxvxg4gXd,https://firebasestorage.googleapis.com/v0/b/ca...,CampusEvent,description,Cahn,,2025-09-01,False,19:00,Theater,user_2wBBT8uelfgS58uEa7r9gfPGZO6
2aKTJZGeCTI1ZXGt7H4T,https://firebasestorage.googleapis.com/v0/b/ca...,New ‘Cats to Old ‘Cats: Farewell Concert,Bienen horn professor emerita Gail Williams (‘...,"Pick-Staiger Concert Hall, 50 Arts Cir Dr Evan...",https://ci.ovationtix.com/36542/performance/11...,2025-05-05,False,19:00,Music,user_2w19KHbyTeAmgZRzeBsEYpUY00b
3RcbYhSKX45WYfmTssWo,https://firebasestorage.googleapis.com/v0/b/ca...,Composer-Performer Presentation: Bec Plexus,"Amsterdam-based composer, vocalist, and produc...",David and Carol McClintock Choral and Recital ...,,2025-04-26,False,14:00,Music,user_2w19KHbyTeAmgZRzeBsEYpUY00b
4hgp2kHwyA9iY3cKkiOl,https://firebasestorage.googleapis.com/v0/b/ca...,"Northwestern University Jazz Ensemble: Mingus,...","Darius Hampton, conductor; Ethan Philion, bass...","Mary B. Galvin Recital Hall, 70 Arts Circle Dr...",https://ci.ovationtix.com/36542/performance/11...,2025-05-06,False,19:30,Music,user_2w19KHbyTeAmgZRzeBsEYpUY00b
FvCKWCSKSF2r0xjUAKtJ,https://firebasestorage.googleapis.com/v0/b/ca...,NU Day @ Wrigley 2025,The annual NU Day @ Wrigley tradition returns!...,"Wrigley Field, 1060 W Addison St, Chicago, Ill...",https://www.mlb.com/cubs/tickets/specials/nort...,2025-05-14,False,18:40,Athletics,user_2w19KHbyTeAmgZRzeBsEYpUY00b
IQFICFY1zDraCSrPWrmy,https://firebasestorage.googleapis.com/v0/b/ca...,"Sandra Cisneros: Reading, Discussion, and Perf...",The Litowitz MFA+MA Program and the Bienen Sch...,"Pick-Staiger Concert Hall, 50 Arts Circle Driv...",https://ci.ovationtix.com/36542/performance/11...,2025-05-07,False,19:00,Guest Speakers,user_2w19KHbyTeAmgZRzeBsEYpUY00b
MZAIRVmiYTWmIdWGp3mz,https://firebasestorage.googleapis.com/v0/b/ca...,As It Is In Heaven,By Arlene Hutton \nMusic by Shaker Village of ...,Josephine Louis Theater 20 Arts Circle Drive,https://ci.ovationtix.com/1771/production/1204473,2025-05-23,False,19:30,Theater,user_2w19KHbyTeAmgZRzeBsEYpUY00b
N39K9SVcS7WvPC2cFxFl,https://firebasestorage.googleapis.com/v0/b/ca...,The Challenge of Reflexivism: Giving Primacy t...,Join us for a conversation with Vijayendra (Bi...,720 University Pl,https://www.eventbrite.com/e/the-challenge-of-...,2025-05-01,False,16:00,Guest Speakers,user_2w19KHbyTeAmgZRzeBsEYpUY00b
PgWmej06SLF70mT08nlF,https://firebasestorage.googleapis.com/v0/b/ca...,party at my place!!!!!!,please come to my house and party with me,my house,,2025-04-25,False,18:00,Comedy/Improv,user_2wBDZAHzpjYKaCk8X0GQPdNCKBX


In [13]:
# ----- HTTP Session with Retries -----
def make_session(retries: int = 3, backoff_factor: float = 0.3) -> requests.Session:
    s = requests.Session()
    retries = Retry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET"],
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    # Set a realistic User-Agent
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; PlanItPurpleBot/1.0; +https://planitpurple.northwestern.edu)"
    })
    return s

In [167]:
def scrape_event_detail(session: requests.Session, detail_path: str) -> Dict:
    """
    Fetches the detail page at detail_path and extracts:
      - 'when'      (the full When: line)
      - 'where'     (the full Where: line)
      - 'category'  (the full Category: line)
      - 'description' (the paragraphs under 'Description:')
    """
    url = BASE_URL + detail_path
    resp = session.get(url, timeout=10)
    resp.raise_for_status()

    dsoup = BeautifulSoup(resp.text, "html.parser")
    wrapper = dsoup.find("div", class_="event_wrapper")
    if not wrapper:
        raise RuntimeError(f"event_wrapper not found on {url}")

    data: Dict[str,str] = {}
    # — New: grab the detail-page title —
    header_div = wrapper.find("div", class_="event_header")
    if header_div:
        h2 = header_div.find("h2")
        if h2:
            data["title"] = h2.get_text(strip=True)

    # 1) Grab all labeled <p> fields
    for p in wrapper.select("p"):
        label_el = p.find("span", class_="label")
        if not label_el:
            continue
        key = label_el.text.strip().rstrip(":").lower()
        full = p.get_text(" ", strip=True)
        # remove label text itself
        value = full.replace(label_el.text, "").lstrip(": ").strip()
        data[key] = value

    # 2) Pull the Description block
    desc = []
    header = dsoup.find(lambda t: t.name in ("h3","h4") and "Description" in t.text)
    if header:
        for sib in header.find_next_siblings():
            if sib.name == "p":
                desc.append(sib.get_text(" ", strip=True))
            else:
                break
    data["description"] = "\n".join(desc).strip()

    # 3) Extract the image URL
    img_div = wrapper.select_one("div.detail.image_detail")
    if img_div:
        img = img_div.find("img", src=True)
        if img:
            data["link_to_image"] = urljoin(BASE_URL, img["src"])

    # 4) Extract the "More Info" / ticket link
    more_info = dsoup.find("a", class_="detail-button more-info", href=True)
    if more_info:
        data["link_to_ticket"] = urljoin(BASE_URL, more_info["href"])

    return data
def scrape_all_events(session: requests.Session, url: str) -> List[Dict]:
    """
    1) Reads the main_event_list
    2) Grabs each <li> → title, date, time & detail-path
    3) Follows detail-path to enrich with when/where/category/description
    """
    resp = session.get(url, timeout=10)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    container = soup.find("div", id="main_event_list")
    if not container:
        raise RuntimeError("div#main_event_list not found")

    events: List[Dict] = []
    current_date = None

    for child in container.find_all(recursive=False):
        if child.name == "h3":
            current_date = child.get_text(strip=True)
        elif child.name == "ul":
            for li in child.find_all("li", recursive=False):
                # 1) detail link
                a = li.find("a", href=True)
                detail_path = a["href"] if a else None

                # 2) time 
                time     = a.get_text(strip=True) if a else None

                base = {
                    "date":    current_date,
                    "time":    time,
                }

                # 3) fetch detail page
                if detail_path:
                    try:
                        detail_data = scrape_event_detail(session, detail_path)
                    except Exception as e:
                        detail_data = {"when": None, "where": None, "category": None, "description": None}
                        print(f"⚠️ failed to fetch details for {detail_path}: {e}")

                    base.update(detail_data)

                events.append(base)

    return events

In [169]:
# ——— Print Events ———
session = make_session()
BASE_URL = "https://planitpurple.northwestern.edu"
all_events = scrape_all_events(session, BASE_URL + "/")

df = pd.DataFrame(all_events)

# strip the prefixes
df['where']   = df['where'].str.replace(r'^Where:\s*',    '', regex=True)
df['category']= df['category'].str.replace(r'^Category:\s*','', regex=True)

# 1) Rename your columns to match Firestore
df = df.rename(columns={
    "date":            "Date",
    "time":            "Time",
    "title":           "EventName",       # or "detail_title" if you prefer the detail‐page title
    "link_to_image":   "LinktoImage",
    "link_to_ticket":  "LinktoTickets",
    "where":           "Location",
    "category":        "Category",
    "description":     "Description",
})

# 2) Convert Date → YYYY-MM-DD
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y-%m-%d")

# 3) Convert Time → 24h HH:MM
#    Coerce errors (e.g. “ALL DAY” → NaT) then back to strings
df["Time"] = (
    pd.to_datetime(df["Time"], format="%I:%M %p", errors="coerce")
      .dt.strftime("%H:%M")
)

# 4) Add the two new constant columns
df["Saved"]  = False
df["UserID"] = None

# 5) Re-order to your exact schema
cols = [
    "Category",
    "Date",
    "Description",
    "EventName",
    "LinktoImage",
    "LinktoTickets",
    "Location",
    "Saved",
    "Time",
    "UserID",
]
df = df[cols]

# 6) Display it
df

,Category,Date,Description,EventName,LinktoImage,LinktoTickets,Location,Saved,Time,UserID
0,"Fine Arts , \n Social",2025-04-29,Exhibition on view at Dittmar April 11 - May 6...,DITTMAR x SITE Galleries Collaborative Exhibition,https://planitpurple.northwestern.edu/uploads/...,https://www.northwestern.edu/norris/arts-recre...,"Norris University Center, Norris University Ce...",False,NaN,None
1,"Multicultural & Diversity , \n ...",2025-04-29,Northwestern University is home to many Native...,Native American & Indigenous Voices at Northwe...,NaN,NaN,"University Library, Book Nook, 1970 Campus Dri...",False,NaN,None
2,"Multicultural & Diversity , \n ...",2025-04-29,"Free and open to all\nMar 20–Jul 19, 2025\nAt ...",Native Pop! At the Newberry,NaN,https://www.newberry.org/calendar/native-pop,NaN,False,NaN,None
3,Fitness & Recreation,2025-04-29,This class begins with quiet reflection and a ...,Sunrise Yoga,NaN,https://nurecreation.com/sports/groupx/schedule,"Henry Crown Sports Pavilion, Studio 2, 2311 Ca...",False,07:00,None
4,Academic,2025-04-29,Radiation Oncology Weekly Resident Conference,Radiation Oncology Weekly Resident Conference,NaN,NaN,Northwestern Memorial Hospital Galter Pavilion...,False,08:00,None
5,Lectures & Meetings,2025-04-29,Cell & Developmental Biology (CDB) Trainee Sem...,CDB Trainee Seminar Series-Bhuvanasundar Ranga...,https://planitpurple.northwestern.edu/uploads/...,NaN,"Robert H Lurie Medical Research Center, Simpso...",False,09:00,None
6,Lectures & Meetings,2025-04-29,Graph Neural Networks (GNNs) are a recent exte...,New Developments in the Theory and Methodology...,NaN,https://www.nitmb.org/theory-and-methodology-o...,"Suite 3500, Chicago, IL 60615",False,09:00,None
7,Other,2025-04-29,Human Resources and various campus partners ha...,White Light Lamps,NaN,https://northwestern.az1.qualtrics.com/jfe/for...,"1914 Sheridan Road, Quiet Room, Room 302 , Eva...",False,09:00,None
8,Other,2025-04-29,Human Resources and various campus partners ha...,White Light Lamps,NaN,https://northwestern.az1.qualtrics.com/jfe/for...,"Henry Crown Sports Pavilion, Wellness Suite, 2...",False,09:00,None
9,Other,2025-04-29,Human Resources and various campus partners ha...,White Light Lamps,NaN,https://northwestern.az1.qualtrics.com/jfe/for...,"1936 Sheridan Road, Room 205, Evanston, IL 602...",False,09:00,None
